# ATP Synthase with Ayush's Tx attempt 1
Tried to create a SBML file, didn't work with general propensity


---

In [18]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

Loading BokehJS ...

Ayush's Code

In [3]:
class transcription_temperature(Mechanism):
    # G + RNAP -->(k1) G:RNAP(closed) -->(k2) G:RNAP(open)----->(k3)   G+RNAP+mRNA
    #Set the name and mechanism_type
    def __init__(self, name="transcription_temperature", rnap="RNAP", mechanism_type="transcription", **keywords):
        if isinstance(rnap, Species):
            self.rnap = rnap
        elif isinstance(rnap, str):
            self.rnap = Species(name=rnap, material_type="protein")
        elif isinstance(rnap, Component) and rnap.get_species() != None:
            self.rnap = rnap.get_species()
        else:
            raise ValueError(
                "'rnap' parameter must be a string or a Component with defined "
                "get_species(), or a chemical_reaction_network.Species object")
        Mechanism.__init__(self, name=name, mechanism_type=mechanism_type, **keywords)
     
    #Overwrite update_species
    def update_species(self, dna, return_rnap = True, transcript = None, **keywords):
        
        species = [dna]
        if return_rnap:
            species += [self.rnap]
        
        closedcomplex = ComplexSpecies([dna,self.rnap])
        opencomplex = Species(name="opencomplex")
        species+= [opencomplex, closedcomplex]
        
        if transcript is None: #Species names can be automatically created
            transcript = Species(dna.name, material_type = "rna")
        species += [transcript]

        return species #it is best to return all species that will be involved in the reactions

    #Overwrite update_reactions
    #This always requires the inputs component and part_id to find the relevant parameters
    def update_reactions(self, dna, component, part_id, transcript = None, **keywords):

        if transcript is None: #Species names should be automatically created the same here as above
            transcript = Species(dna.name, material_type = "rna")
            
        closedcomplex = ComplexSpecies([dna,self.rnap])
        opencomplex = Species(name=dna.name + "_opencomplex")

        
        
        k1 = component.get_parameter("k1", part_id = part_id, mechanism = self)
        k2 = component.get_parameter("k2", part_id = part_id, mechanism = self)
        k3 = component.get_parameter("k3", part_id = part_id, mechanism = self)    
        Temperature= component.get_parameter("Temperature", part_id=part_id, mechanism = self)
        A = component.get_parameter("A", part_id = part_id, mechanism = self)
        B = component.get_parameter("B", part_id = part_id, mechanism = self)
        C = component.get_parameter("C", part_id = part_id, mechanism = self)
        
        
        rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"
               
        reaction_1 = Reaction(inputs = [dna, self.rnap], outputs = [closedcomplex], k = k1, propensity_type = "general",
                              propensity_params= { "rate": rate_string})
        reaction_2 = Reaction(inputs = [closedcomplex], outputs = [opencomplex], k = k2)
        reaction_3 = Reaction(inputs = [opencomplex], outputs = [dna, self.rnap, transcript], k = k3)                           

        #In this case, we just return one reaction
        return [reaction_1, reaction_2, reaction_3]

In [4]:
atpsyn_promoter= Promoter("atpsyn_promoter")

G = DNAassembly("atpsyn", promoter = "atpsyn_promoter")

mech_tx= transcription_temperature()

default_mechanisms={"transcription":mech_tx}

default_parameters ={"k1":0.01, "k2":0.66, "k3":100000, "Temperature":37, "A": 0.0576 , "B": -4.68 , "C": 120 }

M = Mixture("TxTlMixture", components = [G], parameters = default_parameters, mechanisms= default_mechanisms)

CRN = M.compile_crn()

print("Pretty_print representation of the CRN:\n",
      CRN.pretty_print(show_rates = True, show_attributes = True, show_materials = True))

Pretty_print representation of the CRN:
 Species (5) = {0. opencomplex, 1. rna[atpsyn], 2. protein[RNAP], 3. complex[dna[atpsyn]:protein[RNAP]], 4. dna[atpsyn]}
Reactions (3) = [
0. dna[atpsyn] + protein[RNAP] --> complex[dna[atpsyn]:protein[RNAP]]        
        general: k(x)=0.01*(dna_atpsyn*(1/(0.0576*37^2+-4.68*37+120)))
1. complex[dna[atpsyn]:protein[RNAP]] --> atpsyn_opencomplex        
        massaction: k_f(complex[dna[atpsyn]:protein[RNAP]])=0.66*complex[dna[atpsyn]:protein[RNAP]]
2. atpsyn_opencomplex --> dna[atpsyn] + protein[RNAP] + rna[atpsyn]        
        massaction: k_f(atpsyn_opencomplex)=100000*atpsyn_opencomplex
]


In [5]:
temp_vals={25, 37}

timepoints = np.linspace(0, 100, 50)



R = CRN.simulate_with_bioscrape(timepoints = timepoints, 
                                    initial_condition_dict = {"dna_atpsyn":5}, stochastic= True )

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: atpsyn_opencomplex, 
  initial_condition_dict = initial_condition_dict)


In [19]:
p = bokeh.plotting.figure(width = 450, height = 300,
                         x_axis_label = 'time',
                         y_axis_label = 'rna molecules')

palette = colorcet.b_glasbey_category10

p.line(timepoints, R['rna_atpsyn'], legend_label = 'atp synthase rna', line_width = 2, color = palette[0])
p.line(timepoints, R['rna_atpsyn'], legend_label = 'atp synthase rna', line_width = 2, color = palette[0])


p.legend.location = 'top_left'

bokeh.io.show(p)

## Test different temperatures
---

In [62]:

temp_vals=[1, 25, 37, 50, 100]
timepoints = np.linspace(0, 100, 50)
mech_tx= transcription_temperature()
default_mechanism={"transcription":mech_tx}

p = bokeh.plotting.figure(width = 550, height = 350,
                         x_axis_label = 'time (sec)',
                         y_axis_label = 'rna molecules')



for i,val in enumerate(temp_vals): 
    
    P1= Promoter("atpsyn_promoter")
   # P2= Promoter("lac_ara_1")
    G1 = DNAassembly("atpsyn", promoter = "atpsyn_promoter", 
                     parameters={"k1":0.01, "k2":0.66, "k3":100000, "Temperature":val, "A": 0.0576 , "B": -4.68 , "C": 120 }
                    , mechanisms= default_mechanism)
#     G2 = DNAassembly("Y", promoter = "P2",
#                      parameters={"k1":0.001, "k2":0.16, "k3":100000, "Temperature":val, "A": 0.0095 , "B": -0.77 , "C": 20 }
#                     , mechanisms= default_mechanism)
    
    M1 = Mixture("TxTlMixture", components = [G1])
    
    CRN1 = M1.compile_crn()
    
    
    R = CRN1.simulate_with_bioscrape(timepoints = timepoints, 
                                    initial_condition_dict = {"dna_atpsyn":5}, stochastic= True )
    
    p.line(timepoints, R['rna_atpsyn'], legend_label = f'rna at temp {val} C', line_width = 2, color = palette[i])
    
p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

bokeh.io.show(p)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: atpsyn_opencomplex, 
  initial_condition_dict = initial_condition_dict)
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: atpsyn_opencomplex, 
  initial_condition_dict = initial_condition_dict)
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: atpsyn_opencomplex, 
  initial_condition_dict = initial_condition_dict)
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uni

## Write SBML file at 25C
---

In [64]:

timepoints = np.linspace(0, 100, 50)
mech_tx= transcription_temperature()
default_mechanism={"transcription":mech_tx}

p = bokeh.plotting.figure(width = 550, height = 350,
                         x_axis_label = 'time (sec)',
                         y_axis_label = 'rna molecules')



val = 25
    
P1= Promoter("atpsyn_promoter")
   # P2= Promoter("lac_ara_1")
G1 = DNAassembly("atpsyn", promoter = "atpsyn_promoter", 
                     parameters={"k1":0.01, "k2":0.66, "k3":100000, "Temperature":val, "A": 0.0576 , "B": -4.68 , "C": 120 }
                    , mechanisms= default_mechanism)
#     G2 = DNAassembly("Y", promoter = "P2",
#                      parameters={"k1":0.001, "k2":0.16, "k3":100000, "Temperature":val, "A": 0.0095 , "B": -0.77 , "C": 20 }
#                     , mechanisms= default_mechanism)
    
M1 = Mixture("TxTlMixture", components = [G1])
    
CRN1 = M1.compile_crn()

CRN1.write_sbml_file("tx_25C.xml")
    
R = CRN1.simulate_with_bioscrape(timepoints = timepoints, 
                                    initial_condition_dict = {"dna_atpsyn":5}, stochastic= True )
    
p.line(timepoints, R['rna_atpsyn'], legend_label = f'rna at temp {val} C', line_width = 2, color = palette[i])
    
p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

bokeh.io.show(p)

NotImplementedError: SBML writing of general propensities not implemented

In [52]:
%reload_ext watermark
%watermark -v -p biocrnpyler,bioscrape

CPython 3.7.7
IPython 7.13.0

biocrnpyler 0.2.1
bioscrape 1.0.0
